In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import Ipynb_importer
from utils import *
import re

In [2]:
from copy import deepcopy
def co_nodes(route1, route2):
    nodes1 = [(node[0], node[1]) for node in route1]
    nodes2 = [(node[0], node[1]) for node in route2]
    if len(nodes2) == 1:
        return False
    for node in nodes1:
        if node in nodes2:
            return True
    return False

def combine(routes, infos, clues):
    rst_routes = []
    all_info = []
    rst_flag = False   ### true means all clues are found, false means not all but the most clues are found
    for info in infos:
        all_info.extend(info)
    all_info = set(all_info)
    if all_info == set(clues):
        rst_flag = True

    for i in range(len(routes) - 1):
        rst_routes = []
        rst_routes.append(routes[i])
        rst_info = set(infos[i])
        tmp_nodes = deepcopy(routes[i])
        for j in range(i+1, len(routes)):
            # print(rst_routes)
            # print(routes[j])
            flag = co_nodes(tmp_nodes, routes[j])
            if flag:
                rst_routes.append(routes[j])
                rst_info = rst_info.union(infos[j])
                tmp_nodes.extend(routes[j])
        
        if rst_info == all_info:
            break
    return rst_routes, rst_flag


In [ ]:
import json
if __name__ == '__main__':
    # knowledge_labels = get_all_labels()

    data_path = ''


    datas = prepare_dataset(data_path)

    count = 0
    for data in datas:
        count+=1
        llm_call = 1
        question = data['RawQuestion']
        # question = data['question']
        print('question:', question)
        
        # Modify question if the first word is 'where', 'when', or 'why'
        first_word = question.split()[0]
        if first_word.lower() in ['where', 'when', 'who']:
            if first_word.lower() == 'where':
                question = ' '.join(['which place', *question.split()[1:]])
            elif first_word.lower() == 'when':
                question = ' '.join(['what time', *question.split()[1:]])
            elif first_word.lower() == 'who':
                question = ' '.join(['which person', *question.split()[1:]])

        
        ##--------------------------------------------------------------------------------------------------------------
        try :
            clue_chains = extract_entity_chain(question)
            print('clue_chains:', clue_chains)
            if clue_chains[0] in ['place', 'time', 'person']:
                clue_chains = clue_chains[1:] + clue_chains[:1]
            

        except Exception as e:
            print(e)
            continue
        print('clues:', clue_chains)
        init_routes = []
        init_info = []
        
        possible_routes = [] # 如果找不到下个节点（this_info为空），并且还有别的信息没找到，就保存到这里，最后一个tuple里存的是没找到的信息。
        possible_info = []
        
        

        for entity in clue_chains:
            nodes = query_node_by_name(entity)
            if nodes:
                for node in nodes:
                        init_routes.append([node])
                        init_info.append([entity])
                        break
        direct_answer = 0
        end_flag = 0
        if init_routes == []:
            print('no nodes found')
            direct_answer = 1
        depth = 0
        end_flag = 0
        current_routes = []
        for i in range(len(init_routes)):
         if end_flag == 1:
             break
         invovled_info = []
         current_routes = []
         current_routes.append(init_routes[i])
         invovled_info.append(init_info[i])
         while depth < len(clue_chains) and direct_answer == 0:
            if llm_call > 30:
                print('!!!!!!!!!!!!!!!!!!!!!!! llm_call: ', llm_call)
                break
            depth += 1    
            
            new_routes = []
            new_info = []
            for i in range(len(current_routes)): 
               if llm_call > 30:
                   print('!!!!!!!!!!!!!!!!!!!!!!! llm_call: ', llm_call)
                   break     
               try:    
                route = current_routes[i] # 
                # print('route:', route)
                current_info = invovled_info[i] #对应于route的信息
                print('current_info:', current_info)
                related_nodes = get_related_nodes_by_id(route[-1][2]) # 每个节点都是(label, name, id, relation)
                print('related_nodes of ', route[-1][1], ": ",related_nodes)
                if len(route) > 1:
                    related_nodes = [node for node in related_nodes if node[2] != route[-2][2]]
                candidates = [] #候选节点，不包含name完全匹配的节点，要送给大模型选择的
                current_flag = 0
                for related_node in related_nodes:  ### (label, name, id, relation)
                    #先字符匹配，name完全匹配优先级最高，没有的话直接剪枝
                    if related_node[1] in clue_chains and related_node[1] not in current_info:
                        new_routes.append(route+[related_node])
                        tmp_info = current_info+[related_node[1]]
                        current_flag = 1
                        print('name matched:', related_node[1])
                        if related_node[0] in clue_chains and related_node[0] not in current_info:
                            tmp_info.append(related_node[0])
                        if related_node[3] in clue_chains and related_node[3] not in current_info:
                            tmp_info.append(related_node[3])
                        tmp_info = list(set(tmp_info))
                        new_info.append(tmp_info)

                if current_flag == 1:
                    continue

                clue_2_relation = {}
                # 仍然没有找到匹配的节点，就利用大模型进行剪枝
                if current_flag == 0:
                    extracted_nodes = [r for _, _, _, r in related_nodes]
                    # pruned_nodes = [relation, clue, score]
                    extracted_nodes = list(set(extracted_nodes))
                    
                    mapped_nodes = []
                    
                    while len(extracted_nodes) > 20:
                        tmp = relation_mapping(question, list(set(clue_chains)-set(current_info)), (route[-1][0], route[-1][1]), extracted_nodes[:20])
                        llm_call += 1
                        mapped_nodes.extend(tmp)
                        extracted_nodes = extracted_nodes[20:]
                    tmp = relation_mapping(question, list(set(clue_chains)-set(current_info)), (route[-1][0], route[-1][1]), extracted_nodes)
                    mapped_nodes.extend(tmp)
                    print('mapped_nodes: ', mapped_nodes)
                    llm_call += 1
                    for node in mapped_nodes:
                        tmp = []
                        for related_node in related_nodes:
                            if related_node[0] == node[0]:
                                candidates.append(related_node)
                                tmp.append(related_node)
                        if tmp != []:
                            clue_2_relation[node[1]] = clue_2_relation.get(node[1], []) + tmp

                
                if candidates != []:
                        tmp_candi = deepcopy(candidates)
                        tmp_candi = [(cand[0], cand[1]) for cand in tmp_candi]
                        mapped_nodes = []
                        if len(tmp_candi) > 80:
                            tmp_candi = tmp_candi[:80]
                        while len(tmp_candi) > 20:
                            temp = tmp_candi[:20]
                            tmp_candi = tmp_candi[20:]  
                            selected_nodes = entity_mapping(question, list(set(clue_chains)-set(current_info)), temp)
                            mapped_nodes.extend(selected_nodes)
                            llm_call += 1
                            print('llm_call:', llm_call)
                        selected_nodes = entity_mapping(question, list(set(clue_chains)-set(current_info)), tmp_candi)
    
                        llm_call += 1
                        print('llm_call:', llm_call)
                        mapped_nodes.extend(selected_nodes)
                        print('mapped_nodes:', mapped_nodes)

                        for e_node in mapped_nodes:
                            exsit_flag = 0
                            for k, v in clue_2_relation.items():
                                for node in v:
                                    # print('node:', node)
                                    # print('e_node:', e_node)
                                    if node[0] == e_node[0][0] and node[1] == e_node[0][1]:
                                        exsit_flag = 1
                                        new_routes.append(route+[node])
                                        if e_node[1] == k:
                                            new_info.append(current_info+[k])
                                        else:
                                            new_info.append(current_info+[e_node[1], k])
                            if exsit_flag == 0:
                                mapped_nodes.remove(e_node)

                        if len(mapped_nodes) == 0 and len(set(clue_chains)-set(current_info)) == 1:
                            ### 如果只剩一个线索，且上面的关系映射有这个线索而实体映射没有，考虑是多选题的情况
                            for k, v in clue_2_relation.items():
                                if k == list(set(clue_chains)-set(current_info))[0]:
                                    for node in v:
                                        new_routes.append(route+[node])
                                        new_info.append(current_info+[k])
                        elif len(mapped_nodes) == 0:
                            possible_routes.append(route)
                            possible_info.append(current_info)

                else:
                    possible_routes.append(route)
                    possible_info.append(current_info)
                        

               except Exception as e:
                    print(e)
                    continue 
            if new_routes != []:
                current_routes = []
                for i in range(len(new_info)):
                    if set(new_info[i]) == set(clue_chains):
                        end_flag = 1
                        current_routes.append(new_routes[i])
                if end_flag == 0:
                    current_routes = new_routes
                    invovled_info = new_info
            else:
                print('clues undiscovered')
                break

            if end_flag == 1:
                break
        # print('possible:', possible_routes)
        # print('current:', current_routes)

        if end_flag == 0 and direct_answer == 0: ### 如果没有找到所有的线索，就把可能的线索和信息都合并起来
            all_routes = current_routes + possible_routes
            all_infos = invovled_info + possible_info
            current_routes, flag = combine(all_routes, all_infos, clue_chains)

        info_2_llm = []
        for route in current_routes:
            if len(route) == 1:
                info_2_llm.append((route[0][1], None, None))
            else:
                tmp = [(route[i][1], route[i+1][3], route[i+1][1]) for i in range(len(route)-1)]
                for t in tmp:
                    if t not in info_2_llm:
                        info_2_llm.append(t)
                

        if len(info_2_llm) > 30:
            info_2_llm = info_2_llm[:30]
        print(info_2_llm)


        try:
            
            answer_gpt = answer_question(question, info_2_llm)
            if end_flag == 1:
                answer_all_clues = answer_gpt
            else:
                answer_all_clues = chat_llama('please answer the question step by step:' + question)
            llm_call += 1
        except Exception as e:
            answer_gpt = e


        result3 = {
            'data': data,
            'answer': answer_gpt,
            'llm_call': llm_call,
            'answer_all_clues': answer_all_clues
        }


        with open('./results/webqsp.json', 'a', encoding='utf-8') as file:
            try:
                json.dump(result3, file, ensure_ascii=False)
            except Exception as e:
                print(e)
                print(result3)
            file.write('\n')
            file.write(str(info_2_llm))
            file.write('\n')
        print(count, 'done')
